In [1]:
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from IPython.display import JSON

Which data stores are available?

In [2]:
JSON({e.name: e.metadata for e in find_data_store_extensions()})

<IPython.core.display.JSON object>

Get a specific store

Usually we need more information to get the actual data store object. Which data store parameters are available?

In [ ]:
get_data_store_params_schema('directory')

Just provide mandatory parameters to instantiate the store class:

In [ ]:
store = new_data_store('directory', base_dir='../serve/demo')
store

Which datasets are provided? (the list may contain both gridded and vector datasets):

In [ ]:
JSON(list(store.get_data_ids()))

Get more info about a dedicated dataset. This include a description of the possible open formats:

In [ ]:
store.describe_data('cube-1-250-250.zarr')

Which "openers" are available? Note, the first part of an opener's name identifies the returned in-memory data type:

- `dataset` --> `xarray.Dataset` (Gridded data)
- `mldataset` -->  `xcube.core.mlds.MultiLevelDataset`  (Gridded data, multi-resolution pyramid)
- `geodataframe` --> `geopandas.GeoDataFrame` (Vector data) 

In [ ]:
store.get_data_opener_ids()

Which parameters must I pass or are available to open the dataset?

In [ ]:
store.get_open_data_params_schema('cube-1-250-250.zarr')

There are no required parameters, so we can open the dataset without any parameters:

In [ ]:
dataset = store.open_data('cube-1-250-250.zarr')
dataset

In [ ]:
dataset.kd489.isel(time=3).plot.imshow(figsize=(16, 16))